In [1]:
import pandas

import re

from tqdm import tqdm

splitwords_re = re.compile(r"[a-zA-zА-Яа-я]+")

## Читаем корпус новостей из задания 1

In [2]:
news = pandas.read_csv('news.csv', sep=';')
news = news[['content']]
news.head()

,content
0,"Двукратный олимпийский чемпион, американец Май..."
1,"""Объединенная авиастроительная корпорация"" (ОА..."
2,"Китай предупредил Францию, чтобы та не продава..."
3,В Чувашии на базе 42 школ создадут Центры обра...
4,Вылетевший из Санкт-Петербурга в Уфу самолет а...


In [3]:
news['content'][1]

'"Объединенная авиастроительная корпорация" (ОАК), в которую входит корпорация "Иркут", перенесла сроки импортозамещения компонентов для российских самолетов Sukhoi Superjet 100 (SSJ 100) с 2024 г. на 2023 г., заявил вице-премьер Юрий Борисов. Изменение срока предусмотрено\xa0программой\xa0финансового оздоровления ОАК, которая принята правительственной комиссией 12 мая. В свою очередь "Иркут" в апреле объявил\xa0тендер на разработку, изготовление и поставку прототипов системы шасси для SSJ 100 в рамках программы импортозамещения. "Иркут"\xa0объявил также тендеры\xa0на разработку и поставку систем и компонентов для Sukhoi Superjet New. Речь идет о гидравлической и кислородной системах, системах энерго- и водоснабжения, пассажирских креслах и облицовочных панелях. '

## Определяем функции подсчета расстояния Дамерау-Левенштейна и построения триграмм

In [4]:
def damerau_levenshtein_distance(s1, s2):   
    d = {}
    lenstr1 = len(s1)
    lenstr2 = len(s2)
    for i in range(-1,lenstr1+1):
        d[(i,-1)] = i+1
    for j in range(-1,lenstr2+1):
        d[(-1,j)] = j+1
 
    for i in range(lenstr1):
        for j in range(lenstr2):
            if s1[i] == s2[j]:
                cost = 0
            else:
                cost = 1

            d[(i,j)] = min(
                           d[(i-1,j)] + 1, # deletion
                           d[(i,j-1)] + 1, # insertion
                           d[(i-1,j-1)] + cost, # substitution
                          )

            if i and j and s1[i]==s2[j-1] and s1[i-1] == s2[j]:
                d[(i,j)] = min (d[(i,j)], d[i-2,j-2] + cost) # transposition
 
    return d[lenstr1-1,lenstr2-1]


def make_trigrams(word):
    word = f"__{word}"
    trigrams = [word[index: index+3] for index in range(len(word))]
    last = trigrams[-1]
    if len(last) < 3:
        trigrams[-1] += '_' * (3 - len(last) % 3)
    return trigrams

### Строим словарь, где ключ - набор триграмм, а значение это набор слов

In [5]:
dictionary = dict()

for n in tqdm(news['content']):
    all_words = re.findall(splitwords_re, n)

    for word in all_words:
        word = word.lower()
        trigrams = make_trigrams(word)
        
        key = tuple(trigrams)
        dictionary[key] = word

100%|██████████| 15000/15000 [00:21<00:00, 693.59it/s] 


### Функция поиска N рекомендаций

In [6]:
def find_recommend(word, dictionary=dictionary, N=5):
    word = word.lower()
    word_trigram = make_trigrams(word)
    word_trigram = tuple(word_trigram)

    if word_trigram in dictionary:
        return dictionary[word_trigram]
    else:
        top_N = dict()

        for key in dictionary:
            distance = damerau_levenshtein_distance(word_trigram, key)

            if len(top_N) < N:
                top_N[distance] = key
            else:
                for dist in top_N:
                    if distance < dist:
                        top_N.pop(dist)
                        top_N[distance] = dictionary[key]
                        break
        return top_N
    
    return None

### Проверим работоспособность на нескольких словах с опечатками

In [7]:
find_recommend('Ассоциации')

'ассоциации'

In [8]:
find_recommend('Ассоциаци')

{2: 'ассоциации', 3: 'ассоциацию', 4: 'ассоциациям', 7: 'асси', 8: 'асали'}

In [9]:
find_recommend('Асоциации')

{2: 'ассоциации', 5: 'ассоциацию', 6: 'ампутации', 7: 'овации', 9: 'асаду'}

In [10]:
find_recommend('приел')

{3: 'прицел', 4: 'приквел', 6: 'приедешь', 7: 'тассу'}

In [11]:
find_recommend('При_етствие')

{3: 'приветствие',
 5: 'препятствие',
 6: 'приветствия',
 8: 'православие',
 9: 'приезде'}

In [12]:
find_recommend('При___ствие')

{5: 'приветствие',
 8: 'православие',
 6: 'препятствие',
 9: 'протирание',
 10: 'пририсованы'}

In [13]:
find_recommend('переподвыверт')

{8: 'переполняет',
 9: 'переполнением',
 10: 'пересадят',
 11: 'переспросил',
 14: 'приедешь'}